In [1]:
import numpy as np

PARAM_NAMES = ("alpha", "a", "b", "c", "d", "e")
SAFE_POLY_EPS = 1e-12
LOG_RATIO_CLIP = 500.0
POLY_RATIO_CLIP = 1e12
POSITION_GAUSSIAN_REG_STRENGTH = 10
POSITION_GAUSSIAN_REG_SIGMA = 100


In [2]:
def positions_array(x, reg_strength=POSITION_GAUSSIAN_REG_STRENGTH, reg_sigma=POSITION_GAUSSIAN_REG_SIGMA):
    coords = np.asarray(x, dtype=float)
    scalar_input = coords.ndim == 0
    coords = coords.reshape(-1)
    if reg_strength:
        coords = gaussian_l2_regularize(coords, reg_strength, reg_sigma)
    return coords, scalar_input


def gaussian_l2_regularize(coords, strength, sigma):
    if strength <= 0.0:
        return coords
    sigma = float(sigma)
    if sigma <= 0.0:
        raise ValueError('Gaussian regularization sigma must be positive.')
    gaussian_weights = np.exp(-0.5 * (coords / sigma) ** 2)
    return coords / (1.0 + strength * gaussian_weights)


def amplitude_prefactor(coords, prefactor_coeffs):
    a, b, c, d, e = prefactor_coeffs
    return (((e * coords + d) * coords + c) * coords + b) * coords + a


def prefactor_prime(coords, prefactor_coeffs):
    return ((4.0 * prefactor_coeffs[4] * coords + 3.0 * prefactor_coeffs[3]) * coords + 2.0 * prefactor_coeffs[2]) * coords + prefactor_coeffs[1]


def prefactor_second(coords, prefactor_coeffs):
    return (12.0 * prefactor_coeffs[4] * coords + 6.0 * prefactor_coeffs[3]) * coords + 2.0 * prefactor_coeffs[2]


def safe_prefactor(values):
    values = np.asarray(values, dtype=float)
    return np.where(values >= 0.0, np.maximum(values, SAFE_POLY_EPS), np.minimum(values, -SAFE_POLY_EPS))


def log_wave_amplitude(x, wave_params):
    wave_params = np.asarray(wave_params, dtype=float)
    alpha = wave_params[0]
    prefactor_coeffs = wave_params[1:]
    coords, scalar_input = positions_array(x)
    prefactor = amplitude_prefactor(coords, prefactor_coeffs)
    log_prefactor = np.log(np.clip(np.abs(prefactor), SAFE_POLY_EPS, None))
    result = log_prefactor - alpha * coords**2
    return result[0] if scalar_input else result


def local_energy(x, wave_params, anharmonic_coupling):
    wave_params = np.asarray(wave_params, dtype=float)
    alpha = wave_params[0]
    prefactor_coeffs = wave_params[1:]
    coords, scalar_input = positions_array(x)

    prefactor = amplitude_prefactor(coords, prefactor_coeffs)
    safe_pref = safe_prefactor(prefactor)
    prefactor_prime_vals = prefactor_prime(coords, prefactor_coeffs)
    prefactor_second_vals = prefactor_second(coords, prefactor_coeffs)

    ratio_prime = np.clip(prefactor_prime_vals / safe_pref, -POLY_RATIO_CLIP, POLY_RATIO_CLIP)
    ratio_second = np.clip(prefactor_second_vals / safe_pref, -POLY_RATIO_CLIP, POLY_RATIO_CLIP)
    kinetic = -0.5 * (
        ratio_second
        - 2.0 * alpha
        - 4.0 * alpha * coords * ratio_prime
        + 4.0 * (alpha * coords) ** 2
    )
    potential = (anharmonic_coupling* coords**4) + 0.5*coords**2
            #anharmonic_coupling * (16.0 * coords**4 - 48.0 * coords**2 + 12.0) + 0.5*coords ** 2)
    result = kinetic + potential
    return result[0] if scalar_input else result


def log_wave_derivatives(x, wave_params):
    wave_params = np.asarray(wave_params, dtype=float)
    prefactor_coeffs = wave_params[1:]
    coords, scalar_input = positions_array(x)
    safe_pref = safe_prefactor(amplitude_prefactor(coords, prefactor_coeffs))

    derivatives = np.empty((coords.size, len(PARAM_NAMES)))
    derivatives[:, 0] = -coords**2
    powers = np.vstack([np.ones_like(coords), coords, coords**2, coords**3, coords**4]).T
    derivatives[:, 1:] = powers / safe_pref[:, None]
    return derivatives[0] if scalar_input else derivatives



In [3]:
def metropolis_move(positions, wave_params, step_scale, rng):
    positions = np.asarray(positions, dtype=float).copy()
    for idx, current_pos in enumerate(positions):
        proposal = current_pos + step_scale * rng.standard_normal()
        log_ratio = 2.0 * (
            log_wave_amplitude(proposal, wave_params)
            - log_wave_amplitude(current_pos, wave_params)
        )
        accept_prob = min(1.0, np.exp(np.clip(log_ratio, -LOG_RATIO_CLIP, LOG_RATIO_CLIP)))
        if rng.random() < accept_prob:
            positions[idx] = proposal
    return positions


def sr_step(energies, log_grads, wave_params, sr_rate, stabilizer=1e-4):
    energies = np.asarray(energies, dtype=float)
    log_grads = np.asarray(log_grads, dtype=float)
    wave_params = np.asarray(wave_params, dtype=float)

    mean_energy = energies.mean()
    centered_grads = log_grads - log_grads.mean(axis=0)
    energy_fluctuations = energies - mean_energy

    s_tensor = 2.0 * (centered_grads.T @ centered_grads) / len(energies) + stabilizer * np.eye(len(wave_params))
    generalized_force = 2.0 * np.mean(energy_fluctuations[:, None] * centered_grads, axis=0)

    try:
        update = -sr_rate * np.linalg.solve(s_tensor, generalized_force)
    except np.linalg.LinAlgError:
        update = -sr_rate * np.linalg.pinv(s_tensor) @ generalized_force

    energy_uncertainty = energies.std() / np.sqrt(len(energies))
    return wave_params + update, mean_energy, energy_uncertainty


In [4]:
def run_vmc(
    num_particles=1000,
    num_iterations=500,
    warmup_steps=100,
    initial_alpha=0.4,
    sr_rate=1.0,
    step_scale=1.0,
    initial_prefactor=(1.0, 0.0, 0.0, 0.0, 0.0),
    anharmonic_coupling=1.0,
    readout_stride=25,
    random_seed=None,
):
    if warmup_steps >= num_iterations:
        raise ValueError('Warm-up steps must be smaller than the total number of iterations.')

    wave_params = np.array((initial_alpha, *initial_prefactor), dtype=float)
    rng = np.random.default_rng(random_seed)
    positions = rng.standard_normal(num_particles)

    print('--- SR VMC ---')
    print(f'anharmonic coupling = {anharmonic_coupling}')
    param_label = ", ".join(f"{name}={value:.4f}" for name, value in zip(PARAM_NAMES, wave_params))
    print(f'initial wave params: {param_label}')
    print(f"{'iter':>6}  {'energy':>12}  {'sigma':>12}  wave params")

    mean_energy = np.nan
    energy_uncertainty = np.nan

    for iteration in range(num_iterations):
        positions = metropolis_move(positions, wave_params, step_scale, rng)

        if iteration >= warmup_steps:
            energies = local_energy(positions, wave_params, anharmonic_coupling)
            log_grads = log_wave_derivatives(positions, wave_params)
            wave_params, mean_energy, energy_uncertainty = sr_step(
                energies,
                log_grads,
                wave_params,
                sr_rate,
            )

            if iteration % readout_stride == 0:
                param_label = ", ".join(f"{name}={value:.4f}" for name, value in zip(PARAM_NAMES, wave_params))
                print(
                    f"{iteration:6d}  {mean_energy:12.6f}  {energy_uncertainty:12.6f}  {param_label}"
                )

    param_label = ", ".join(f"{name}={value:.4f}" for name, value in zip(PARAM_NAMES, wave_params))
    print('--- relaxation complete ---')
    print(f'final wave params: {param_label}')
    if np.isfinite(mean_energy):
        print(f'final energy: {mean_energy:.6f} +/- {energy_uncertainty:.6f}')
    else:
        print('not enough samples for a stable energy estimate.')

    return wave_params, mean_energy, energy_uncertainty


In [10]:
if __name__ == '__main__':
    best_wave_params, best_energy, energy_spread = run_vmc(
        num_particles=2000,
        num_iterations=1200,
        warmup_steps=100,
        initial_alpha=0.03,
        sr_rate=0.1,
        step_scale=1.0,
        initial_prefactor=(0.1, 0.1, 0.1, 0.1, 0.1),
        anharmonic_coupling=1,
        readout_stride=20,
        random_seed=0,
    )

    print()
    print('Results summary:')
    param_label = ", ".join(f"{name}={value:.4f}" for name, value in zip(PARAM_NAMES, best_wave_params))
    print(f'Optimized wave params: {param_label}')
    if np.isfinite(best_energy):
        print(f'Energy estimate: {best_energy:.6f} +/- {energy_spread:.6f}')
    else:
        print('Energy estimate unavailable; warm-up window may be too long.')


--- SR VMC ---
anharmonic coupling = 1
initial wave params: alpha=0.0300, a=0.1000, b=0.1000, c=0.1000, d=0.1000, e=0.1000
  iter        energy         sigma  wave params
   100     10.448516      0.541422  alpha=1.5881, a=-0.0532, b=-0.2288, c=-0.1840, d=0.4528, e=0.5132
   120      1.276044      0.083448  alpha=2.2399, a=0.3232, b=0.1802, c=0.3682, d=-1.9507, e=3.0958
   140      0.749525      0.023612  alpha=2.1088, a=1.4241, b=0.2387, c=1.4002, d=-1.6146, e=2.7716
   160      0.815557      0.005613  alpha=1.9715, a=2.3604, b=0.0379, c=2.2573, d=0.0450, e=2.0453
   180      0.806065      0.002431  alpha=1.9202, a=2.4340, b=0.0058, c=2.5117, d=-0.0009, e=1.6462
   200      0.804024      0.000867  alpha=1.8360, a=2.5905, b=0.0013, c=2.5966, d=-0.0009, e=1.2463
   220      0.803369      0.000433  alpha=1.8007, a=2.6506, b=-0.0012, c=2.6047, d=0.0006, e=1.1021
   240      0.804174      0.000611  alpha=1.8256, a=2.6079, b=0.0026, c=2.6139, d=-0.0123, e=1.1842
   260      0.803525      0.

In [6]:
#lambda: 0.0001 E= 0.067
#0.001 E= 0.1269
#0.01 E= 0.1562